In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
import json
from sklearn.metrics import classification_report

In [2]:
import spacy
from spacy.tokens import DocBin
from spacy.lang.en import English
nlp = spacy.blank("en")

## Bagging to BERT: A tour of applied NLP
### Part 2: Beyond bagging
### Table of Contents
* [LSTM](#lstm)
* [BERT](#bert)



### Data processing <a class="anchor" id="data"></a>

Copied from part 1

You'll either need to download the [imdb review data](https://ai.stanford.edu/~amaas/data/sentiment/) and save it to this directory OR download the [processed data](https://drive.google.com/file/d/1oN_fO91IBkDHD_u6WXiUCvhhyNexQDJq/view?usp=sharinghttps://drive.google.com/file/d/1oN_fO91IBkDHD_u6WXiUCvhhyNexQDJq/view?usp=sharing).

In [ ]:
# # processing the original data into DataFrame
# # here for reference, don't need to run this if you're using reviews.pkl.gz
# source_path = Path('./aclImdb/')
# #neg_files = source_path.glob('./*/neg/*.txt')
# #pos_files = source_path.glob('./*/pos/*.txt')
# all_files = []
# for f in source_path.glob('./*/*/*.txt'):
#     filename = f.as_posix()
#     if 'unsup' not in filename:
#         # split up into useful components
#         _, split, sent, idx = filename.split('/')
#         idx = int(idx.split('_')[0])
#         all_files.append([idx, split, sent, f.read_text()])
# review_df = pd.DataFrame(all_files)
# review_df.columns = ['idx', 'split', 'label', 'text']
# # some minor html cruft is in here
# review_df['text'] = review_df['text'].str.replace('<br /><br />', '')
# review_df = review_df.to_pickle('reviews.pkl.gz')

In [ ]:
# can skip here if you already have reviews.pkl.gz
review_df = pd.read_pickle('reviews.pkl.gz')
review_df['label'] = review_df['label'] == 'pos'

In [ ]:
# copied from part 1: using the same train/test split
seed = 37
np.random.seed(seed)
pct_train = 0.7
X_train, X_test, y_train, y_test = train_test_split(
    review_df['text'],
    review_df['label'], train_size=pct_train)

In [ ]:
# you may need to restart after this in Collab
!pip install spacy-transformers

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
!git clone -b revised_2023 https://github.com/bpben/bagging_to_bert

Cloning into 'bagging_to_bert'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 72 (delta 36), reused 53 (delta 17), pack-reused 0
Unpacking objects: 100% (72/72), 27.39 MiB | 11.39 MiB/s, done.


In [3]:
def preprocess(text, labels, name):
  
    # initialize spaCy's DocBin format (easier use with spaCy pipeline)
    db = DocBin()
    output_path = f'{name}.spacy'
    data_tuples = zip(text, labels)
    # pipe is slightly faster than individually processing each
    for doc, label in nlp.pipe(data_tuples, as_tuples=True):
      # store the labels in the document's .cat attribute
      if label == 'pos':
            doc.cats['pos'] = True
            doc.cats['neg'] = False
        else:
            doc.cats['pos'] = False
            doc.cats['neg'] = True
        db.add(doc)
    # save the DocBin
    db.to_disk(output_path)

In [8]:
# spaCy default corpus reader has certain expectations about format
review_df = pd.read_pickle(
    '/content/drive/MyDrive/talks/odsc_2023/reviews.pkl.gz')
# copied from part 1: want to use the same train/test split
seed = 37
np.random.seed(seed)
pct_train = 0.7
X_train, X_test, y_train, y_test = train_test_split(
    review_df['text'],
    review_df['label'], train_size=pct_train)

In [6]:
# running the preprocessing on each split
preprocess(X_train, y_train, 'train')
preprocess(X_test, y_test, 'test')

In [3]:
from spacy.cli.train import train

# can override config info with overrides
# the tutorial config file doesn't have the paths for train/dev corpora
# going to just run this for a few epochs, see how it works
train("bagging_to_bert/spacy_materials/config_trf.cfg",
      output_path='example_model',
      overrides={"paths.train": "train.spacy", 
                 "paths.dev": "test.spacy",
                 "training.max_epochs": 1},
      use_gpu = 0)

ℹ Saving to output directory: example_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'textcat']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  -------------  ------------  ----------  ------
  0       0           0.00          0.25       35.15    0.35
  0     200           0.00         50.14       36.27    0.36
  0     400           0.00         40.94       76.70    0.77
  0     600           0.01         33.34       70.19    0.70
  0     800           0.03         36.88       76.52    0.77
  0    1000           0.06         41.10       82.04    0.82
  0    1200           0.05         32.34       82.00    0.82
  0    1400           0.05         28.28       83.54    0.84
  0    1600           0.07         33.07       83.66    0.84
  0    1800           0.09         28.94       84.20    0.84
  0    2000           0.09         30.79       74.64    0.75
  0    2200           0.16         29.76

In [5]:
# load the best version of the model
m = spacy.load('/content/drive/MyDrive/talks/odsc_2023/trained_trf')

In [6]:
# looking at the outputs from running a simple example
m('This movie is great').cats

{'pos': 0.9958682060241699, 'neg': 0.004131876397877932}

SpaCy has its own evaluation capabilities, but for comparison's sake, let's use the same evaluation approach we did with out other applications.

In [ ]:
# if you're interested in trying out spacy's own evaluate
#from spacy.cli.evaluate import evaluate
#evaluate(model='./example_model/model-best/', data_path='test.spacy')

In [ ]:
# get the predicted category from the model
pred = np.array([max(d.cats, key=d.cats.get) for d in m.pipe(X_test)])

In [ ]:
print(f'accuracy: {np.where(pred == y_test)[0].shape[0]/y_test.shape[0]}')
print(
    classification_report(y_pred=pred,
                          y_true=y_test))

### LSTM <a class="anchor" id="lstm"></a>


In [ ]:
class SentLSTM(nn.Module):
# adapted from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/
    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(SentLSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # The linear layer that maps from hidden state space to sentiment space
        self.fc = nn.Linear(hidden_dim, output_size)

        # sigmoid activiation
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # forward pass of nn
        batch_size = x.shape[0]
        # this is fit during training
        embeds = self.word_embeddings(x)
        lstm_out, _ = self.lstm(embeds)
        # from lstm space to prediction space
        pred_space = self.fc(lstm_out)
        out = self.sigmoid(pred_space)
        # reshape - want to get the prediction at end of seq
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out

def doc_to_index(docs, vocab, tokenizer, unknown=1):
    # transform docs into series of indices
    docs_idxs = []
    for d in docs:
        w_idxs = []
        d_tokenized = tokenizer(d)
        for w in d_tokenized:
            if w in vocab:
                w_idxs.append(vocab[w])
            else:
                # unknown if not in vocab
                w_idxs.append(unknown)
        docs_idxs.append(w_idxs)
    return(docs_idxs)

def pad_sequence(seqs, seq_len=200):
    # function for adding padding to ensure all seq same length
    features = np.zeros((len(seqs), seq_len),dtype=int)
    for i, seq in enumerate(seqs):
        if len(seq) != 0:
            features[i, -len(seq):] = np.array(seq)[:seq_len]
    return features

In [ ]:
# need to adapt vocab, leave space for padding
tfidf = TfidfVectorizer(stop_words='english', min_df=0.01)
tfidf.fit(X_train)
vocab = tfidf.vocabulary_
tokenizer = tfidf.build_tokenizer()
# need to add "special" tokens
vocab = dict([(v, vocab[v]+2) for v in vocab])
vocab['_UNK'] = 1
vocab['_PAD'] = 0
# from documents to vocab index
parsed_train = doc_to_index(X_train, vocab, tokenizer)
padded_train = pad_sequence(parsed_train)
parsed_test = doc_to_index(X_test, vocab, tokenizer)
padded_test = pad_sequence(parsed_test)


In [ ]:
# construct datasets for loading by PyTorch
train_data = TensorDataset(torch.from_numpy(padded_train), 
                           torch.from_numpy(y_train.values))
test_data = TensorDataset(torch.from_numpy(padded_test), 
                          torch.from_numpy(y_test.values))

batch_size = 100

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,
                         drop_last=True) # this is to keep the size consistent
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,
                        drop_last=True)

In [ ]:
# binary problem, single output
model_params = {'output_size': 1,
               'hidden_dim': 512,
               'embedding_dim': 400,
               'vocab_size': len(vocab)}
model = SentLSTM(**model_params)
model.to(device)

In [ ]:
# learning rate
lr=0.005
# binary cross-entropy loss
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# increasing this will make the training take a while on CPU
epochs = 1
counter = 0
print_every = 5
# gradient clipping: https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    for inputs, labels in train_loader:
        counter += 1
        inputs, labels = inputs.to(device), labels.to(device)
        # sets gradients to zero for each batch
        model.zero_grad()
        output = model(inputs)
        # pred vs actual
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        # print progress
        if counter%print_every == 0:
            val_losses = []
            model.eval()
            for inp, lab in test_loader:
                inp, lab = inp.to(device), lab.to(device)
                out = model(inp)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

In [ ]:
# pytorch LSTM model
model.load_state_dict(torch.load('./state_dict.pt'))
model.eval()
pred_collect = np.array([])
eval_loader = DataLoader(test_data, batch_size=batch_size)
# eval mode
with torch.no_grad():
    for inputs, labels in eval_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs)
        # takes output, rounds to 0/1
        pred = torch.round(output.squeeze())
        pred_collect = np.concatenate([
          pred_collect,
          pred.cpu().numpy()
        ])

In [ ]:
print(f'accuracy: {np.where(pred_collect == y_test)[0].shape[0]/y_test.shape[0]}')
print(
    classification_report(y_pred=pred_collect,
                          y_true=y_test))

### BERT <a class="anchor" id="bert!pip install transformers"></a>
From [HF tutorials](https://huggingface.co/blog/sentiment-analysis-python).  The sentiment analysis pipeline packages together the tokenizer and the BERT model with a classification layer.  The default pipeline uses this [distilBERT model](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english). 

In [ ]:
# this will need to be run if you don't already have this package
#!pip install transformers

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", truncation=True, padding=True)

In [ ]:
# some manipulations for speed and to play nice with BERT
bert_pred = sentiment_pipeline(X_test.apply(lambda x: x).head(n=50).tolist())
bert_pred = [p['label']=='POSITIVE' for p in bert_pred]

In [ ]:
np.where(bert_pred == y_test[:50])
print(f'accuracy: {np.where(bert_pred == y_test[:50])[0].shape[0]/50}')
print(
    classification_report(y_pred=bert_pred,
                          y_true=y_test[:50]))

There it is - you've leveraged a cutting edge model to do sentiment analysis! This performance is pretty good, but our count vectors actually did a few points better.  Maybe there's an opportunity to fine-tune the BERT model specifically to the IMDB review dataset.  Let's try it.

** IN PROGRESS **

Still have been having some issues - performance is conspicuously low.  Use with caution.

NOTE: This takes some time to run, even with the Collab GPU.  You might want to experiment on subsets of the dataset

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# WIP - "num_labels" seems like it should be 1, have seen 2 (pos/neg)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",  num_labels=1)


In [ ]:
def preprocess_function(example):
    return tokenizer(example, truncation=True)

class IMDbDataset(torch.utils.data.Dataset):
    # comes from https://huggingface.co/transformers/v3.4.0/custom_datasets.html
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.astype(float)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# we will subsample because it makes this faster
n = 1000
tokenized_train = tokenizer(X_train.tolist()[:n], 
                            truncation=True, padding=True)
tokenized_test = tokenizer(X_test.tolist()[:n], 
                           truncation=True, padding=True)

train_dataset = IMDbDataset(tokenized_train, y_train.values[:n])
test_dataset = IMDbDataset(tokenized_test, y_test.values[:n])

In [ ]:
# using HF's trainer
# again, this comes from https://huggingface.co/transformers/v3.4.0/custom_datasets.html

training_args = TrainingArguments(
   output_dir='./',
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=5,
   weight_decay=0.01,
   save_strategy="epoch",
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
   tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
# hacky - moving back to cpu where the other data is located
# HF suggests a "prediction" Trainer
m = model.to(torch.device('cpu'))

In [ ]:
# same parameters we trained with
classifier_pipeline = pipeline(task="text-classification", 
                               model=m, tokenizer=tokenizer,
                               truncation=True, padding=True)

In [ ]:
# pred outputs slightly different from sentiment pipeline
bert_pred = classifier_pipeline(X_test.tolist())
bert_pred = [p['score']>0.5 for p in bert_pred]

In [ ]:
# performance is pretty bad, even after 5 epochs
print(f'accuracy: {np.where(bert_pred == y_test)[0].shape[0]/len(y_test)}')
print(
    classification_report(y_pred=bert_pred,
                          y_true=y_test))